<a href="https://colab.research.google.com/github/mzijewel/TensorflowExercise/blob/master/Supervised/Classifcation/Hand%20Writing/HandWritingPyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import dependecies
import torch
from torch import nn,save,load
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

from PIL import Image

In [4]:
# get data
train=datasets.MNIST(root="data",download=True,train=True,transform=ToTensor())
dataset=DataLoader(train,32)
# 1,28,28 - classes (0-9)


In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.model=nn.Sequential(
            nn.Conv2d(1,32,(3,3)),
            nn.ReLU(),
            nn.Conv2d(32,64,(3,3)),
            nn.ReLU(),
            nn.Conv2d(64,64,(3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6),10) # used 3 conv2d, so reduced 3x2 = 6

        )
    def forward(self,x):
        return self.model(x)


In [8]:
# device='cuda' # cuda -> if gpu is available
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=Model().to(device)
opt=Adam(model.parameters(),lr=0.01)
loss_fn=nn.CrossEntropyLoss()

In [9]:
epochs=5

for epoch in range(epochs):
    for batch in dataset:
        x,y=batch
        x,y=x.to(device),y.to(device)
        y_pred=model(x)
        loss=loss_fn(y_pred,y)

        opt.zero_grad()
        loss.backward()
        opt.step()

    print(f"epchs: {epoch} - loss: {loss.item()}")

epchs: 0 - loss: 0.07405541092157364
epchs: 1 - loss: 0.058793094009160995
epchs: 2 - loss: 0.0039445627480745316
epchs: 3 - loss: 0.022062506526708603
epchs: 4 - loss: 0.0637168139219284


In [14]:
# Evaluation function
def evaluate_model(model, test_loader, loss_fn, device):
   model.eval()  # Set the model to evaluation mode
   total_correct = 0
   total_samples = 0
   total_loss = 0.0


   with torch.no_grad():
       for inputs, labels in test_loader:
           inputs, labels = inputs.to(device), labels.to(device)


           # Forward pass
           outputs = model(inputs)
           loss = loss_fn(outputs, labels)


           # Accumulate loss
           total_loss += loss.item()


           # Calculate accuracy
           _, predicted = torch.max(outputs, 1)
           total_samples += labels.size(0)
           total_correct += (predicted == labels).sum().item()


   accuracy = total_correct / total_samples
   average_loss = total_loss / len(test_loader)


   print(f'Test Accuracy: {accuracy * 100:.2f}%')
   print(f'Average Test Loss: {average_loss:.4f}')

In [15]:
# Create a model with trained data
model=Model()
model.load_state_dict(load('model.pt',map_location=device))

<All keys matched successfully>

In [16]:
# test data
test_data=datasets.MNIST(root="data",download=False,train=False,transform=ToTensor())
test_dataset=DataLoader(test_data,32)

evaluate_model(model,test_dataset,loss_fn,device)

Test Accuracy: 92.14%
Average Test Loss: 0.3396


In [ ]:
# save model
save(model.load_state_dict(),'model.pt')

In [21]:
# test with external image
img=Image.open('images/3.png').convert("L").resize((28,28))
img_tensor=ToTensor()(img).unsqueeze(0).to(device)


In [22]:
p=model(img_tensor)
q=torch.argmax(p)
q

tensor(3)